In [ ]:
import pandas as pd

In [ ]:
df_JP = pd.read_excel("ryotaro_filled.xlsx",sheet_name=1)

In [ ]:
# Removing the first row & getting rid of the ID-type column
df_JP = df_JP.iloc[1:].reset_index(drop = True).drop('ryotaro', axis = 1)
# Fixing column names
df_JP.columns = ['abstract','contribution','category']

In [ ]:
df_JP.head()

,abstract,contribution,category
0,集中治療室（ICU）で人工呼吸器を装着している心臓血管外科術後患者にタクティールケアを実施し...,本論文は、集中治療室（ICU）で人工呼吸器を装着している心臓血管外科術後患者にタクティールケ...,Analytical
1,症例は 55 歳女性．X−2 年から両肩と両股関節痛を認め，当院総合内科紹介で精査を行われた...,本論文は、再発性多発性軟骨炎を持つ55歳女性が気管気管支軟化症を発症した事例を報告し、本疾患...,Case study
2,43 歳男性、心窩部痛，黄疸。X 年 3 月に肝酵素上昇を伴う腹痛のため近医を受診，胆 嚢炎...,本論文は、43歳男性に対して内視鏡的乳頭部切除術（endoscopic papillecto...,Case study
3,骨髄増殖性疾患（MPN）において血小板数が増加し偽性高 K 血症となることは知られている．今...,本論文では、慢性骨髄性白血病（CML）患者において，血小板増多により偽性高 K 血症に加え，...,Case study
4,有害物質であるフェノールを用いない迅速簡便なアンモニア態窒素測定法の確立を目指し，連続流れ分...,本論文では、従来法に代替可能で、かつ有害物質であるフェノールを用いない迅速簡便なアンモニア態...,Methodological


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load SeamlessM4T model
model_name = "facebook/seamless-m4t-v2-large"  # or a smaller model if available
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from tqdm import tqdm
import re

def translator(rawText,src_lang,tgt_lang):
    #Construct tokenized input with source language.
    #Split with period. Use lookbehind to make sure not to split by decimal period (like 10.1)
    sentences = re.split(r'(?<=[^0-9])\.', rawText)
    print(len(sentences))

    translatedSentences =""
    for sentence in sentences:
      #Skip if it is an empty sentence
      if sentence == "" or sentence == " ":
        continue
      print(sentence)

      #Push tokenized input to GPU
      input = tokenizer(
          sentence, return_tensors="pt",src_lang=src_lang
      ).to('cuda')

      #Translate to target language
      with torch.no_grad():
          output = model.generate(**input, tgt_lang=tgt_lang)

      #Decode to translated text
      translatedSentence = tokenizer.decode(output[0], skip_special_tokens=True)
      print(translatedSentence)

      # Clear input and output tensors from memory
      del input
      del output
      torch.cuda.empty_cache()

      translatedSentences += " " + translatedSentence

    #Return translated text
    return translatedSentences

def japaneseParser(text):
  #There shouldn't be any whitespace in Japanese
  #Existence of whitespace is confirmed to affect the quality of translation
  text = text.replace(' ','')
  #Japanese sentence might contain wider white space
  text = text.replace('　','')
  #Replace "Japanese period" with English period for sentence tokenization
  text = text.replace('。','.')
  #There happen to be Japanese version of English period...
  text = text.replace('．','.')

  #Return parsed text
  return text

def translateDataFrame(df, src_lang, tgt_lang):
  abstracts = df['abstract']
  contributions = df['contribution']

  translatedAbstractList = []
  translatedContributionList = []

  for i in tqdm(range(len(df))):
    abstract_raw = abstracts[i]
    contribution_raw = contributions[i]

    #Replace newline with white space if there's any.
    abstract_raw.replace('\n',' ')
    contribution_raw.replace('\n',' ')

    #Japanese usually doesn't include empty space. Replace with "".
    #Existence of empty space would decrease the accuracy of translation.
    if src_lang == "jpn":
      abstract_raw = japaneseParser(abstract_raw)
      contribution_raw = japaneseParser(contribution_raw)
      print("Japanese parsed")

    #Retrieve translated text for both abstract and contribution
    abstract_translated = translator(abstract_raw,src_lang,tgt_lang)
    contribution_translated = translator(contribution_raw,src_lang,tgt_lang)

    #Append translatedn text to list
    translatedAbstractList.append(abstract_translated)
    translatedContributionList.append(contribution_translated)
  #Construct DataFrame with translated abstracts and translated contributions
  translatedDF = pd.DataFrame({'abstract_translated':translatedAbstractList,'contribution_translated':translatedContributionList})
  #Append category series from original DataFrame
  translatedDF['category'] = df['category']

  return translatedDF


In [ ]:
df_JPtoEN = translateDataFrame(df_JP, "jpn", "eng")

  0%|          | 0/10 [00:00<?, ?it/s]

Japanese parsed
7
集中治療室（ICU）で人工呼吸器を装着している心臓血管外科術後患者にタクティールケアを実施し，安静時と比較してストレス緩和の効果を検証した
Tactile care was administered to post-operative patients in the intensive care unit (ICU) wearing ventilators, and the effects of stress relief were tested compared to quiet time.
ICUに入室した心臓血管外科術後の人工呼吸器装着者を対象とした
It was aimed at people who had been admitted to the ICU with artificial respiratory devices after cardiovascular surgery.
病室内で，同一対象者に，仰臥位での安静後，タクティールケアを実施，再び安静とし，その前後に主観的および客観的指標のデータを取得した
In the ward, the same subject was given tactile care after being sedated in a reclining position, then sedated again, and subsequently obtained data on subjective and objective indicators.
分析対象者は4名（全員男性，平均年齢64.3±10.9歳）で，タクティールケア中・後にバイタルサインの大きな変動はなかった
The subjects were four men, with an average age of 64.3 years and 10.9 months, and there was no significant change in vital signs during or after tactile care.
客観的指標である血中オキシトシンや血中コルチゾールでは共通した変化は見られなかったが，主観的指標である不安の指標State-TraitAnxietyInventory-FormJYZ（新版STAI）得点

 10%|█         | 1/10 [00:17<02:37, 17.53s/it]

Tactile care has been suggested to be safe during the post-operative invasive injury period, to reduce subjective pain, anxiety, to have a relaxing effect, and to lead to stress relief.
Japanese parsed
14
症例は55歳女性
The case is a 55-year-old woman.
X−2年から両肩と両股関節痛を認め，当院総合内科紹介で精査を行われたが確定診断に至らず，対症療法で経過観察となった
He was diagnosed with shoulder and hip joint pain from X-2 and underwent a comprehensive internal medicine examination at the hospital, but was not diagnosed and underwent a follow-up examination in anti-seizure therapy.
X−1年からぶどう膜炎が認められて当院眼科で加療を受けた
I've been diagnosed with vitiligo since I was a year old, and I've been treated at the eye hospital.
X年6月に人間ドックの肺機能検査で混合性換気障害を指摘され，7月に当科を紹介受診した
In June of the same year, he was diagnosed with mixed ventilatory failure in a human dock lung examination, and in July, he was introduced to the department.
数日後に労作時呼吸困難が出現し予約外受診となり，気道狭窄音を聴取した
A few days later, he had difficulty breathing during his labor, was admitted out of the ordinary, and heard 

 20%|██        | 2/10 [00:41<02:49, 21.17s/it]

...

In this paper, we examined the effects of sublingual immunotherapy (SLIT) on psoriasis and rhinitis, and concluded that the effects of SLIT on psoriasis were high even in years of mass dispersion, and the effects increased year by year, but rhinitis had worsening symptoms due to the visual analogy scale (VAS) year by year.
Japanese parsed
5
メラノーマの治療を中心に飛躍的な進歩を遂げた2010年以降の薬物療法は,今なお治療成績の向上をめざした開発競争のなかにある
The advances in drug therapy since 2010 have made a breakthrough in the treatment of melanoma, and it's still in the midst of a development race to improve treatment outcomes.
殺細胞性抗癌剤の時代から分子標的治療や免疫療法へと移行しており,皮膚がんにおける分子標的治療としては,メラノーマのBRAF遺伝子変異例を対象としたBRAF阻害薬とMEK阻害薬の併用療法がよく知られている
The transition from the era of cell-killing anticancer drugs to molecular targeted therapies and immunotherapies has led to the well-known combination of BRAF inhibitors and MEK inhibitors for melanoma BRAF gene mutations as a molecular targeted treatment for skin cancer.
免疫療法においては,メラノーマやメルケル細胞癌,有棘細胞癌をはじめとした上皮系皮膚悪性腫

 90%|█████████ | 9/10 [03:04<00:16, 16.06s/it]

This paper introduces new drug treatments for melanoma and discusses changes in the future treatment system through early implementation of cancer genetic panel testing.
Japanese parsed
6
現在サイクリン依存性キナーゼ4/6(以下,CDK4/6)阻害薬の導入により治療成績は向上してきたが,CDK4/6阻害薬に起因すると考えられる死亡例を含む重篤な間質性肺疾患も相次いで報告されている
Although the introduction of the current cyclin-dependent kinase 4-4-6 (CDK6) inhibitor has improved treatment outcomes, severe interstitial lung disease, including deaths thought to be caused by CDK-4-6 inhibitors, has also been reported.
症例は78歳女性,6年前から乳がんに対する治療を受けていたが右鎖骨上窩リンパ節に再発し,放射線照射とフルベストラント+パルボシクリブを開始
The case is a 78-year-old woman who had been treated for breast cancer for six years, but had a relapse in the right clavicle lymph node and began radiation therapy with full-busted and parvo-script.
発熱・呼吸困難等で緊急受診し右肺に浸潤影～すりガラス影,左肺底部の浸潤影を認めた
I'm going to put a little bit of salt in it, and I'm going to put a little bit of salt in it.
2回目のステロイド・パルス療法により肺野陰影・呼吸困難の改善が得られた
The second round of steroid pulse

100%|██████████| 10/10 [03:15<00:00, 19.54s/it]

This paper presents cases of severe interstitial lung disease in which a cyclin-dependent kinase 4 6 (CDK 4 6) inhibitor was used to treat breast cancer.


In [ ]:
df_JPtoEN.to_csv('papers_JPtoEN.csv')

In [ ]:
df_ES = pd.read_excel("jose.xlsx",sheet_name=1)

In [ ]:
df_EStoEN = translateDataFrame(df_ES, "spa", "eng")

  0%|          | 0/10 [00:00<?, ?it/s]

9
Los incendios forman parte del ciclo de algunos ecosistemas, los cuales pueden ser causantes de la degradación de otros
Fires are part of the cycle of some ecosystems, which can cause the degradation of others.
 Algunas de sus causas son principalmente antropogénicas, entre ellas las fogatas mal apagadas, actividades de agricultura y liberación de basura, que generan la pérdida de hábitats y contaminación aérea a gran escala
Some of its causes are mainly anthropogenic, including poorly extinguished fires, agricultural activities and garbage disposal, which generate habitat loss and large-scale air pollution.
 El presente trabajo hace referencia a la aplicación de un algoritmo genético basado en una colonia de hormigas para generar, de forma teórica, las rutas de verificación para el monitoreo y la detección temprana de incendios forestales en el Estado de México mediante dispositivos aéreos no tripulados, debido a que es una de las entidades con mayor número de este tipo de incidenci

 10%|█         | 1/10 [00:15<02:20, 15.61s/it]

This study proposes a new method for the early detection of forest fires in the State of Mexico, using genetic algorithms and drones to optimize monitoring routes, which contributes significantly to the prevention and control of these events.
9
El programa computacional para la modelación de la falla por volcamiento en dos dimensiones (2D), llamado volteoLab, permite interpretar el modo de falla tipo volcamiento (toppling en inglés) de un conjunto de bloques de roca en un talud de carretera bajo el análisis de cargas de contacto y fuerzas gravitacionales en un marco de análisis
The computational program for the modeling of the fault by overturning in two dimensions (2D), called volteoLab, allows to interpret the fault mode type overturning (toppling in English) of a set of rock blocks on a road slope under the analysis of contact loads and gravitational forces in an analysis framework
 VolteoLab ha sido realizado a partir del método de equilibrio límite
VolteoLab has been carried out u

 20%|██        | 2/10 [00:34<02:21, 17.75s/it]

...

Addressing these challenges is critical to maximizing the potential of AI and improving people's health
3
Tratamiento de datos para la calificación de vinos, este informe detalla el proceso seguido, en donde se utilizó el lenguaje de programación Phyton, para el análisis de los datos del dataset, se utilizó el servidor Google Colab para ejecutar los algoritmos en la nube ya que el equipo considero que la velocidad de análisis de datos en google colab es más rápido
The data processing for the wine rating, this report details the process followed, where the Phyton programming language was used for the analysis of the dataset data, the Google Colab server was used to run the algorithms in the cloud since the team considers that the speed of data analysis in Google Colab is faster
 Las redes neuronales tienen capacidad de aprender y realizar tareas basadas en un entrenamiento inicial llamado aprendizaje adaptativo y además de que son tolerantes a los fallos
Neural networks have the ability

 90%|█████████ | 9/10 [02:06<00:12, 12.75s/it]

This study describes using Google Colab to run the algorithms in the cloud, which is faster than the team in person, for wine rating.
3
La seguridad informática, es un área muy importante en cualquier sistema que tenga conexión a internet, debido a que existen sitios Web fraudulentos que pueden realizar acciones delictivas hacia una persona, organización u otra entidad
Computer security is a very important area in any system that has an internet connection, because there are fraudulent websites that can carry out criminal actions against a person, organization or other entity
 Por lo cual es necesario poder detectar qué sitios web son fraudulentos antes de poder ingresar a ella, para ello se desarrolló una implementación mediante Árboles de Decisión con el lenguaje de Python para poder detectar y clasificarlos en Legítimos, Sospechosos y Fraudulentos por medio de 1353 casos que clasifican a los sitios webs
Therefore, it is necessary to be able to detect which websites are fraudulent be

100%|██████████| 10/10 [02:13<00:00, 13.36s/it]

The Commission is also aware of the fact that the European Parliament has been calling for such a ban for a long time.


In [ ]:
df_EStoEN.to_csv('papers_EStoEN.csv')